In [2]:
"""
LANDFIRE assessments for MODIS and VIIRS AFDs western U.S.

Data sources:
    - LANDFIRE Existing Vegetation Type (EVT) proportional cover
    - LANDFIRE Canopy Bulk Density (CBD) mean
    - LANDIFRE Canopy Base Height (CBH) mean
    - LANDFIRE Canopy Cover (CC) percentage
    
Author: maxwell.cook@colorado.edu
"""

import os, gc
import geopandas as gpd
import rioxarray as rxr

import warnings
warnings.filterwarnings("ignore") # suppresses annoying geopandas warning

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [4]:
# Load the combined MODIS and VIIRS AFDs (buffered)
fp = os.path.join(projdir,f'data/spatial/mod/AFD/combined-afd_aspen-fires_2018_to_2023_buffer.gpkg')
afds = gpd.read_file(fp)
afds.columns

Index(['LATITUDE', 'LONGITUDE', 'BRIGHTNESS', 'SCAN', 'TRACK', 'ACQ_DATE',
       'ACQ_TIME', 'SATELLITE', 'INSTRUMENT', 'CONFIDENCE', 'VERSION',
       'BRIGHT_T31', 'FRP', 'DAYNIGHT', 'TYPE', 'VID', 'NIFC_ID', 'NIFC_NAME',
       'START_YEAR', 'DISCOVERY_DATE', 'WF_CESSATION_DATE', 'na_l3name',
       'ACQ_MONTH', 'ACQ_YEAR', 'ACQ_DATETIME', 'afdID', 'geometry'],
      dtype='object')

In [ ]:
# Tidy columns for simplicity.
afds = afds[['afdID','START_YEAR','geometry']]

### Calculate the Proportional EVT in AFD Observations

From LANDFIRE EVT (ca. 2016) calculate the proportional cover within AFD observations (buffers). 

In [ ]:
# Load the EVT raster

In [5]:
evt_fp = os.path.join(maindir,'data/landcover/LANDFIRE/LF2016_EVT_200_CONUS/Tif/LC16_EVT_200.tif')
evt_da = rxr.open_rasterio(evt_fp, masked=True, cache=False, chunks='auto').squeeze()
shp, gt, wkt, nd = evt_da.shape, evt_da.spatial_ref.GeoTransform, evt_da.rio.crs, evt_da.rio.nodata
print(
    f"Shape: {shp}; \n"
    f"GeoTransform: {gt}; \n"
    f"WKT: {wkt}; \n"
    f"NoData Value: {nd}; \n"
    f"Data Type: {evt_da[0].dtype}")
gc.collect()

Shape: (97283, 154207); 
GeoTransform: -2362425.0 30.0 0.0 3177435.0 0.0 -30.0; 
WKT: EPSG:5070; 
NoData Value: nan; 
Data Type: float32


81

In [ ]:
# Crop to fire bounds.


In [ ]:
# Run zonal statistics for the AFD buffers

In [ ]:
start = time.time()    

# Read in the LANDFIRE layers
cbd_path = os.path.join(maindir,'data/landcover/LANDFIRE/LF2020_CBD_200_CONUS/Tif/LC20_CBD_200.tif')
cbh_path = os.path.join(maindir,'data/landcover/LANDFIRE/LF2020_CBH_200_CONUS/Tif/LC20_CBH_200.tif')
tcc_path = os.path.join(maindir,'data/landcover/LANDFIRE/LF2020_CC_200_CONUS/Tif/LC20_CC_200.tif')

lf_paths = [cbd_path, cbh_path, tcc_path]

results = []
for lf in lf_paths:
    print(f'Processing {os.path.basename(lf)}')
    name = os.path.basename(lf).split('_')[1]
    
    # Calculate the zonal statistics
    df = calc_zonal_stats(vectors=frp_plots[['VID','geometry']], raster_path=lf, attr=name)
    
    results.append(df)

print(f"Total elapsed time: {round((time.time() - start)/60)} minutes")